# Finjustering av Open AI-modeller

Denne notatboken er basert på den nåværende veiledningen i [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst)-dokumentasjonen fra Open AI.

Finjustering forbedrer ytelsen til grunnmodeller for din applikasjon ved å trene den opp igjen med ekstra data og kontekst som er relevant for det spesifikke bruksområdet eller scenariet. Merk at prompt engineering-teknikker som _few shot learning_ og _retrieval augmented generation_ lar deg forbedre standardprompten med relevant data for å øke kvaliteten. Disse tilnærmingene er imidlertid begrenset av maks antall tokens som foundation-modellen støtter.

Med finjustering trener vi faktisk opp selve modellen med nødvendig data (slik at vi kan bruke langt flere eksempler enn det som får plass i maks token-vindu) – og vi får en _tilpasset_ versjon av modellen som ikke lenger trenger å få eksempler ved inferens. Dette gjør ikke bare promptdesignet vårt mer effektivt (vi får mer fleksibilitet til å bruke token-vinduet til andre ting), men kan også redusere kostnadene våre (ved å redusere antall tokens vi må sende til modellen ved inferens).

Finjustering har 4 steg:
1. Forbered treningsdataene og last dem opp.
1. Kjør treningsjobben for å få en finjustert modell.
1. Evaluer den finjusterte modellen og iterer for kvalitet.
1. Ta i bruk den finjusterte modellen for inferens når du er fornøyd.

Merk at ikke alle foundation-modeller støtter finjustering – [sjekk OpenAI-dokumentasjonen](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) for oppdatert informasjon. Du kan også finjustere en modell som allerede er finjustert. I denne veiledningen bruker vi `gpt-35-turbo` som vår foundation-modell for finjustering.

---


### Steg 1.1: Forbered datasettet ditt

La oss lage en chatbot som hjelper deg å forstå periodesystemet ved å svare på spørsmål om et grunnstoff med et limerick. I _denne_ enkle veiledningen skal vi bare lage et datasett for å trene modellen med noen få eksempler på svar som viser forventet format på dataene. I et virkelig scenario må du lage et datasett med mange flere eksempler. Du kan også bruke et åpent datasett (for ditt fagområde) hvis det finnes, og omformatere det for bruk til finjustering.

Siden vi fokuserer på `gpt-35-turbo` og ønsker et enkelt svar (chat fullføring), kan vi lage eksempler ved å bruke [dette foreslåtte formatet](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) som følger OpenAI sine krav for chat fullføring. Hvis du forventer samtaler med flere meldinger, bør du bruke [formatet for flere meldinger](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) som inkluderer en `weight`-parameter for å angi hvilke meldinger som skal brukes (eller ikke) i finjusteringen.

Vi bruker det enklere formatet med én melding for denne veiledningen. Dataene er i [jsonl-format](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) med én post per linje, hver representert som et JSON-objekt. Snutten under viser 2 poster som eksempel – se [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) for hele eksempelsamlingen (10 eksempler) vi bruker i finjusteringsveiledningen. **Merk:** Hver post _må_ defineres på én linje (ikke delt opp over flere linjer slik det ofte gjøres i formatert JSON-fil)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

I et virkelig scenario trenger du et mye større sett med eksempler for gode resultater – avveiningen vil være mellom kvaliteten på svarene og tid/kostnader for finjustering. Vi bruker et lite sett slik at vi kan fullføre finjusteringen raskt og vise prosessen. Se [dette eksempelet fra OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) for en mer avansert veiledning om finjustering.


### Steg 1.2 Last opp datasettet ditt

Last opp dataene ved å bruke Files API [som beskrevet her](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Merk at for å kjøre denne koden, må du først ha gjort følgende:
 - Installert `openai` Python-pakken (sørg for at du bruker versjon >=0.28.0 for de nyeste funksjonene)
 - Satt miljøvariabelen `OPENAI_API_KEY` til din OpenAI API-nøkkel
For mer informasjon, se [oppsettguiden](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) som er laget for kurset.

Kjør nå koden for å opprette en fil for opplasting fra din lokale JSONL-fil.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Steg 2.1: Opprett finjusteringsjobben med SDK-en


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Steg 2.2: Sjekk statusen til jobben

Her er noen ting du kan gjøre med `client.fine_tuning.jobs` API-et:
- `client.fine_tuning.jobs.list(limit=<n>)` - Vis de siste n finjusteringsjobbene
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Hent detaljer om en spesifikk finjusteringsjobb
- `client.fine_tuning.jobs.cancel(<job_id>)` - Avbryt en finjusteringsjobb
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Vis opptil n hendelser fra jobben
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Det første steget i prosessen er _å validere treningsfilen_ for å sikre at dataene har riktig format.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Steg 2.3: Spor hendelser for å følge med på fremdriften


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Trinn 2.4: Se status i OpenAI-dashboardet


Du kan også se statusen ved å besøke OpenAI-nettsiden og gå til _Finjustering_-delen av plattformen. Her får du oversikt over statusen til den nåværende jobben, og du kan også følge historikken for tidligere kjøringer. I dette skjermbildet ser du at den forrige kjøringen feilet, mens den andre kjøringen lyktes. Til sammenligning skjedde dette fordi den første kjøringen brukte en JSON-fil med feil format på postene – etter at dette ble rettet opp, ble den andre kjøringen fullført uten problemer, og modellen ble tilgjengelig for bruk.

![Finjusteringsjobb-status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.no.png)


Du kan også se statusmeldinger og målinger ved å bla lenger ned i det visuelle dashbordet som vist her:

| Meldinger | Målinger |
|:---|:---|
| ![Meldinger](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.no.png) |  ![Målinger](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.no.png)|


### Steg 3.1: Hent ID og test finjustert modell i kode


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Steg 3.2: Last inn og test finjustert modell i Playground

Nå kan du teste den finjusterte modellen på to måter. Først kan du gå til Playground og bruke nedtrekksmenyen for modeller for å velge din nylig finjusterte modell fra listen over alternativer. Den andre muligheten er å bruke "Playground"-valget som vises i finjusteringspanelet (se skjermbilde over), som åpner følgende _sammenligningsvisning_ hvor grunnmodellen og den finjusterte modellen vises side om side for rask evaluering.

![Finjusteringsjobb-status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.no.png)

Fyll ganske enkelt inn systemkonteksten som ble brukt i treningsdataene dine og skriv inn testspørsmålet ditt. Du vil se at begge sidene oppdateres med identisk kontekst og spørsmål. Kjør sammenligningen, så ser du forskjellen i svarene mellom dem. _Legg merke til hvordan den finjusterte modellen gir svaret i det formatet du brukte i eksemplene dine, mens grunnmodellen bare følger systemprompten_.

![Finjusteringsjobb-status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.no.png)

Du vil også se at sammenligningen viser antall tokens for hver modell, samt hvor lang tid inferensen tok. **Dette spesifikke eksempelet er veldig enkelt og ment for å vise prosessen, men gjenspeiler ikke et reelt datasett eller scenario**. Du kan legge merke til at begge eksemplene viser samme antall tokens (systemkontekst og brukerprompt er identiske), men at den finjusterte modellen bruker mer tid på inferensen (egen modell).

I virkelige situasjoner vil du ikke bruke et så enkelt eksempel som dette, men finjustere mot ekte data (for eksempel produktkatalog for kundeservice) hvor kvaliteten på svaret blir mye tydeligere. I _den_ sammenhengen vil det å oppnå tilsvarende svar med grunnmodellen kreve mer tilpasset promptutforming, noe som vil øke tokenbruken og potensielt også tiden det tar å kjøre inferensen. _For å prøve dette, ta en titt på finjusteringseksemplene i OpenAI Cookbook for å komme i gang._



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi tilstreber nøyaktighet, vær oppmerksom på at automatiske oversettelser kan inneholde feil eller unøyaktigheter. Det opprinnelige dokumentet på sitt originale språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell, menneskelig oversettelse. Vi er ikke ansvarlige for eventuelle misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
